### Importing modules

In [219]:
import re
import RAKE
import pymorphy2
import numpy as np
import pandas as pd
from summa import keywords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

### Loading data

Manually grabbed from vesti.ru, not the best source in terms of content but got some accurate tags which after a little clean-up may be considered as keywords.

In [220]:
df = pd.read_csv('corpus.tsv', sep='\t')

In [221]:
df.head()

,title,text,keywords_site,keywords_manual
0,Евросоюз ввел санкции против 8 россиян,Еще восемь россиян и двоих сирийцев Шади и Наз...,"Китай/КНР, Сирия, Алексей Навальный, Евросоюз,...","Евросоюз, Россия, ФСБ, Алексей Навальный, Сири..."
1,Небензя назвал ничтожной резолюцию ООН,"Проект резолюции Генеральной Ассамблеи ООН, ко...","резолюция, Василий Небензя, репарации, Генерал...","Генеральная Ассамблея ООН, Россия, Василий Неб..."
2,Юрий Бутусов покидает театр имени Вахтангова,Главный режиссер театра имени Вахтангова попро...,"театр, Юрий Бутусов, общество, Россия","театр, театр имени Вахтангова, Юрий Бутусов, Р..."
3,Испуганные сотрудники Twitter сравнивают Маска...,Сотрудники Twitter боятся упоминать имя нового...,"технологии, Twitter, hi-tech, общество, соцсети","Twitter, Илон Маск, мессенджер"
4,"МВД: замглавы Херсонской области не пропала, а...",Заместитель главы Херсонской области по цифров...,"происшествия, задержание, общество, Херсонская...","Херсонская область, МВД, экономическое преступ..."


### Preprocessing

In [222]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [223]:
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = []
    for token in text.split(' '):
        token = token.lower()
        if token.isalpha() and token not in stop_words:
            token = morph.normal_forms(token.strip())[0]
            tokens.append(token)
    return ' '.join(tokens)

In [224]:
def normalize(lst):
    return set([preprocess(x) for x in lst if preprocess(x)])

In [225]:
df['text'] = df['title']+' '+df['text']
df['preprocessed'] = df['text'].apply(preprocess)
df['golden'] = (df['keywords_site'].str.split(', ')+df['keywords_manual'].str.split(', ')).apply(normalize)
df = df[['text', 'preprocessed', 'golden']]

In [226]:
df.head()

,text,preprocessed,golden
0,Евросоюз ввел санкции против 8 россиян Еще вос...,евросоюз ввести санкция против россиянин восем...,"{евросоюз, сирия, санкция, жозеп боррель, росс..."
1,Небензя назвал ничтожной резолюцию ООН Проект ...,небензить назвать ничтожный резолюция оон прое...,"{резолюция, генеральный ассамблея оон, киев, у..."
2,Юрий Бутусов покидает театр имени Вахтангова Г...,юрий бутусов покидать театр имя вахтангов глав...,"{юрий бутусов, театр имя вахтангов, театр, рос..."
3,Испуганные сотрудники Twitter сравнивают Маска...,испуганный сотрудник twitter сравнивать маска ...,"{hitech, технология, илона маск, общество, twi..."
4,"МВД: замглавы Херсонской области не пропала, а...",мвд замглавы херсонский область пропасть задер...,"{задержание, херсонский область, владимир саль..."


### Keywords extraction

In [227]:
def drop_freqs(lst):
    return [x[0] for x in lst]

##### RAKE

In [228]:
rake = RAKE.Rake(stop_words)
df['rake'] = df['text'].apply(lambda x: 
                                      rake.run(x, 
                                               maxWords=2, 
                                               minFrequency=1)).apply(drop_freqs).apply(normalize)

##### TextRank

In [229]:
df['textrank'] = df['preprocessed'].apply(lambda x: 
                                                  keywords.keywords(x, 
                                                                    language='russian', 
                                                                    additional_stopwords=stop_words, 
                                                                    scores=True)).apply(drop_freqs).apply(normalize)

##### Tf-Idf

In [230]:
def tfidf_tags(text, top_n):
    vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))
    tfidf = vectorizer.fit_transform([text])
    weights = np.argsort(np.asarray(tfidf.sum(axis=0)).ravel())[::-1]
    feature_names = np.array(vectorizer.get_feature_names())
    return feature_names[weights[:top_n]]

In [231]:
df['tfidf'] = df['preprocessed'].apply(lambda x: 
                                               tfidf_tags(x, 
                                                          top_n=10))

In [232]:
df.head()

,text,preprocessed,golden,rake,textrank,tfidf
0,Евросоюз ввел санкции против 8 россиян Еще вос...,евросоюз ввести санкция против россиянин восем...,"{евросоюз, сирия, санкция, жозеп боррель, росс...","{сотрудник тюрьма, сообщать тасс, страна предл...","{санкционный список, служба, который, сотрудни...","[санкция, санкция против, против, россиянин, п..."
1,Небензя назвал ничтожной резолюцию ООН Проект ...,небензить назвать ничтожный резолюция оон прое...,"{резолюция, генеральный ассамблея оон, киев, у...","{резолюция, цель, который, запад, заморозить, ...","{выплата, который, полянский, рф, легализовать...","[легализовать, резолюция, рф, ничтожный, это, ..."
2,Юрий Бутусов покидает театр имени Вахтангова Г...,юрий бутусов покидать театр имя вахтангов глав...,"{юрий бутусов, театр имя вахтангов, театр, рос...","{театр получить, увольнение, париж, год, поста...","{главный, решение объяснить нахождение, театр ...","[театр, театр имя, имя, режиссёр театр, режисс..."
3,Испуганные сотрудники Twitter сравнивают Маска...,испуганный сотрудник twitter сравнивать маска ...,"{hitech, технология, илона маск, общество, twi...","{илона, гарри поттер, новое сотрудник, твиттер...","{илона, который, сравнивать маска воландеморт,...","[компания, сотрудник, маск, воландеморт, daily..."
4,"МВД: замглавы Херсонской области не пропала, а...",мвд замглавы херсонский область пропасть задер...,"{задержание, херсонский область, владимир саль...","{сообщить, владимир сальдо, уголовный дело, ра...","{херсонский область, пока, последний, мвд, ека...","[область, губарев, мвд, заместитель, задержать..."


### Syntatic templates

Most frequent syntactic templates of keywords seemed to contain nouns and look as follows: NOUN, ADJ+N and NOUN+NOUN. VP-keywords don't make much sense to me, and could hardly notice any on the website, so let us take these.

In [233]:
def pos_extract(keywords):
    tags = []
    for keyword in keywords:
        if len(keyword.split()) == 1:
            tags.append(morph.parse(keyword)[0].tag.POS)
        else:
            tags.append([morph.parse(w)[0].tag.POS for w in keyword.split()])
    return tags

In [234]:
templates = ["NOUN", 
             "['NOUN', 'NOUN']", 
             "['ADJF', 'NOUN']"]

def filter_templates(modes):
    for mode in modes:
        filtered = []
        for keywords in df[mode]:
            keywords = list(keywords)
            tags = pos_extract(keywords)
            filtered.append([keywords[i] for i, tag in enumerate(tags) if str(tag) in templates])
        df[mode+'_filtered'] = filtered

In [235]:
filter_templates(['rake', 'textrank', 'tfidf'])

In [236]:
df.head()

,text,preprocessed,golden,rake,textrank,tfidf,rake_filtered,textrank_filtered,tfidf_filtered
0,Евросоюз ввел санкции против 8 россиян Еще вос...,евросоюз ввести санкция против россиянин восем...,"{евросоюз, сирия, санкция, жозеп боррель, росс...","{сотрудник тюрьма, сообщать тасс, страна предл...","{санкционный список, служба, который, сотрудни...","[санкция, санкция против, против, россиянин, п...","[сотрудник тюрьма, гражданин россия, обвинение...","[санкционный список, служба, сотрудник, компан...","[санкция, россиянин, хоурань, россия, евросоюз..."
1,Небензя назвал ничтожной резолюцию ООН Проект ...,небензить назвать ничтожный резолюция оон прое...,"{резолюция, генеральный ассамблея оон, киев, у...","{резолюция, цель, который, запад, заморозить, ...","{выплата, который, полянский, рф, легализовать...","[легализовать, резолюция, рф, ничтожный, это, ...","[резолюция, цель, запад, слово, покупка оружие]","[выплата, рф]","[резолюция, рф, оон, выплата, репарация]"
2,Юрий Бутусов покидает театр имени Вахтангова Г...,юрий бутусов покидать театр имя вахтангов глав...,"{юрий бутусов, театр имя вахтангов, театр, рос...","{театр получить, увольнение, париж, год, поста...","{главный, решение объяснить нахождение, театр ...","[театр, театр имя, имя, режиссёр театр, режисс...","[увольнение, париж, год, общий сложность, спек...",[бутусов],"[театр, театр имя, имя, режиссёр театр, режисс..."
3,Испуганные сотрудники Twitter сравнивают Маска...,испуганный сотрудник twitter сравнивать маска ...,"{hitech, технология, илона маск, общество, twi...","{илона, гарри поттер, новое сотрудник, твиттер...","{илона, который, сравнивать маска воландеморт,...","[компания, сотрудник, маск, воландеморт, daily...","[илона, гарри поттер, новое сотрудник, непосре...","[илона, сотрудник, компания, корпоративный мес...","[компания, сотрудник, маск, воландеморт, маска..."
4,"МВД: замглавы Херсонской области не пропала, а...",мвд замглавы херсонский область пропасть задер...,"{задержание, херсонский область, владимир саль...","{сообщить, владимир сальдо, уголовный дело, ра...","{херсонский область, пока, последний, мвд, ека...","[область, губарев, мвд, заместитель, задержать...","[владимир сальдо, уголовный дело, екатерина, п...","[херсонский область, мвд, екатерина губарев, с...","[область, губарев, мвд, заместитель, херсонски..."


### Metrics

In [237]:
def metrics(tags):
    golden = df['golden']
    precs, recalls, f_scores = [], [], []
    for i in range(len(golden)):
        precision = len(set(tags[i]).intersection(golden[i]))/len(tags[i])
        precs.append(precision)
        
        recall = len(set(tags[i]).intersection(golden[i]))/len(golden[i])
        recalls.append(recall)
        
        f_score = (2 * precision * recall) / (precision + recall + 0.001)
        f_scores.append(f_score)
        
    return {'Precision': round(sum(precs)/len(precs), 3), 
            'Recall': round(sum(recalls)/len(recalls), 3),
            'F_score': round(sum(f_scores)/len(f_scores), 3)}

#### Rake

In [238]:
print('Without templates:', metrics(df['rake']), 
      '\nWith templates:',  metrics(df['rake_filtered']))

Without templates: {'Precision': 0.083, 'Recall': 0.261, 'F_score': 0.117} 
With templates: {'Precision': 0.148, 'Recall': 0.258, 'F_score': 0.172}


#### TextRank

In [239]:
print('Without templates:', metrics(df['textrank']), 
      '\nWith templates:',  metrics(df['textrank_filtered']))

Without templates: {'Precision': 0.117, 'Recall': 0.145, 'F_score': 0.116} 
With templates: {'Precision': 0.228, 'Recall': 0.133, 'F_score': 0.151}


#### Tf-Idf

In [240]:
print('Without templates:', metrics(df['tfidf']), 
      '\nWith templates:',  metrics(df['tfidf_filtered']))

Without templates: {'Precision': 0.224, 'Recall': 0.287, 'F_score': 0.247} 
With templates: {'Precision': 0.31, 'Recall': 0.278, 'F_score': 0.285}


### Overall

<center>
<table>
<tr>
    <td></td>
    <td></td>
    <th>Rake</th>
    <th>TextRank</th>
    <th>Tf-Idf</th> 
</tr>
<tr>
    <th>Without templates</th>
    <td>Precision</td>
    <td>0.083</td> 
    <td>0.117</td> 
    <td>0.224</td> 
</tr>
<tr>
    <td></td>
    <td>Recall</td>
    <td>0.261</td> 
    <td>0.145</td> 
    <td>0.287</td> 
</tr>
<tr>
    <td></td>
    <td>F_score</td>
    <td>0.117</td> 
    <td>0.116</td> 
    <td>0.247</td> 
</tr>

</table>  

<table>
<tr>
    <td></td>
    <td></td>
    <th>Rake</th>
    <th>TextRank</th>
    <th>Tf-Idf</th> 
</tr>

<tr>
    <th>With templates</th>
    <td>Precision</td>
    <td>0.148</td> 
    <td>0.228</td> 
    <td>0.310</td> 
</tr>
    
<tr>
    <td></td>
    <td>Recall</td>
    <td>0.258</td> 
    <td>0.133</td> 
    <td>0.278</td> 
</tr>

<tr>
    <td></td>
    <td>F_score</td>
    <td>0.172</td> 
    <td>0.151</td> 
    <td>0.285</td> 
</tr>

</table>  
    </center>

Tf-Idf seemed to outperform the two other approaches, both with and without filtering by part-of-speech templates. Using POS-templates led to a better result for every method, which is an expected result as many VP-like constituents, that had been mistakenly recognized by the tested algorithms as keywords, were thrown away.

Still, the scores are not as high as we would like them to be. Let us take a look at some of the constituents that the algorithms recognized as keywords:  

In [166]:
for i, r in enumerate(df['rake'][2:4], start=1):
    print(f'{i}: {r}')

1: {'театр получить', 'увольнение', 'париж', 'год', 'поставить', 'общий сложность', 'спектакль', 'свой решение', 'занимать должность', 'возвращение', 'театр', 'объяснить нахождение', 'россия', 'отсутствие план', 'понедельник'}
2: {'илона', 'гарри поттер', 'новое сотрудник', 'твиттереть', 'выскочить', 'непосредственный руководство', 'сотрудник жаловаться', 'который критиковать', 'ранее сообщаться', 'себя', 'корпоративный мессенджер', 'быть отслеживать', 'случай', 'кто', 'слово', 'переписка', 'называть', 'маск', 'парень наверху', 'новый владелец', 'сообщение', 'компания tesla', 'новость', 'компания', 'сравнивать бизнесмен', 'антагонист', 'сми', 'воландеморт'}


In (1) as keywords were recognized some pretty common words as 'возвращение', 'год', 'занимать должность', which can hardly characterize not only this specific, but pretty much any text. In (2) we see some really accurate keywords such as 'компания tesla', still some common, nonspecific words as 'сми' and useless verbs as 'выскочить' can be found. This applies to every other tested method.

As a result, it seemed clear to me that the tested algorithms keep on extracting VP-constituents and treat them as keywords, which in fact can rarely act as ones. Throwing away these VPs helps achieve a better result. Also, a smarter preprocessing could help extracting better keywords because handy pymorphy seemed to struggle with NPs that have genitive noun as a dependent.